# Import The GraphQL Summit Graph

This python script will scrape the GraphQL Summit schedule website and insert the session data into Neo4j.

![](../images/neo4j.png)

In [ ]:
import urllib
from bs4 import BeautifulSoup
import csv
import requests
import cssutils
from neo4j import GraphDatabase

In [ ]:
url = "https://summit.graphql.com/speakers"
page = requests.get(url).text
soup = BeautifulSoup(page, 'html.parser')

speakers = soup.find_all('div', attrs={'class': 'SpeakersList-module--list--1FaUX'})

talks = []

titles = soup.select('[class*="infoTitle"]')
descriptions = soup.select('[class*="infoText"]')
names = soup.select('[class*="Speaker-module--name"]')
jobs = soup.select('[class*="Speaker-module--text"]')
pics = soup.select('[class*="HoloContainer-module--photo-"]')

#titles = list(set(titles))
#descriptions = list(set(descriptions))


print(len(titles))
print(len(descriptions))
print(len(names))

for i in range(0,len(names)):
    talk = {}
    talk['session_title'] = titles[2*i].text
    talk['description'] = descriptions[2*i].text
    talk['speaker'] = names[i].text
    
    speaker_bio = jobs[i].text.split(" at ")
    
    talk['speaker_title'] = speaker_bio[0]
    talk['company'] = speaker_bio[1]
    
    #print(pics[i]["style"])
    
    style = cssutils.parseStyle(pics[i]["style"])
    
    url = style['background-image']
    
    talk['pic'] = url.replace(url[:4], '').replace(url[:-1], '')
    talks.append(talk)
    
print(talks)



In [ ]:
# session_title, speaker_title, description, speaker, company, pic

query = '''
UNWIND $talks as talk
MERGE (s:Session {title: talk.session_title})
  SET s.description = talk.description
MERGE (sp:Speaker {name: talk.speaker})
  SET sp.pic = talk.pic, sp.title = talk.speaker_title //FIXME: relationship property ;-)
MERGE (c:Company {name: talk.company})
MERGE (s)<-[:PRESENTS]-(sp)
MERGE (sp)-[:WORKS_FOR]->(c)
'''

In [ ]:
from neo4j import *


uri = "bolt://localhost:7687"
user = "neo4j"
password = "letmein"

driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session() as session:
        session.run(query, parameters={'talks': talks})